## Exp Record Editting

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r"D:\ExpRecord_tmp.xlsx")
df_formal = pd.read_excel(r"S:\ExpSpecTable_Augment.xlsx")

In [28]:
def process_concat_cells(df, out_excel, Animal):
    """Process the raw form excel copied from onenote to well formed excel
    Filter the array using `Animal` label""
    """
    if isinstance(df,str):
        df = pd.read_excel(df)
    df = df.dropna(axis=0, how='all') # drop lines full with nan
    df = df.reset_index(drop=True)  # (index=range(df.shape[0]))  # make the index contiguous!
    row_num = df.shape[0]
    #%%
    msk = ~ df.ephysFN.isna()  # 500 rows
    # df.ephysFN[msk].str.contains("Alfa")  # 436 rows containing Alfa
    if Animal is "Alfa":
        search_str = "Alfa|ALfa"
    elif Animal is "Beto":
        search_str = "Beto"
    elif Animal is "Both":
        search_str = "Beto|Alfa|ALfa"
    else:
        search_str = "Beto|Alfa|ALfa"
    ExpEphysNames = df.ephysFN[df.ephysFN.str.contains(search_str)==True]
    RowidEphs = ExpEphysNames.index
    ExpBhv2Names = df.expControlFN[df.expControlFN.str.contains(search_str)==True]
    RowidBhv = ExpEphysNames.index
    assert RowidEphs is RowidBhv
    #%%
    df.comments.fillna(value="", inplace=True)
    df.comments = df.comments.astype("str")
    df.stimuli.fillna(value="", inplace=True)
    df.stimuli = df.stimuli.astype("str")
    #%%
    for Expi, rowi in enumerate(RowidEphs):
        if Expi != len(RowidEphs) - 1:
            nextrow = RowidEphs[Expi + 1]
        else:
            nextrow = row_num
        print("\nExp %d\t %s\t %s"%( Expi, df.ephysFN[rowi], df.expControlFN[rowi]))
        print(df.comments[rowi:nextrow].str.cat(sep="\n"))
    # 
    stimuli_miss_cnt = 0  # Count how many stimuli entries are missed
    df_sort = df[df.ephysFN.str.contains(search_str)==True]
    df_sort = df_sort.reset_index(drop=True)
    for Expi, rowi in enumerate(RowidEphs):
        if Expi != len(RowidEphs) - 1:
            nextrow = RowidEphs[Expi + 1]
        else:
            nextrow = row_num
        df_sort.comments[Expi] = df.comments[rowi:nextrow].str.cat(sep="\n")
        df_sort.ephysFN[Expi] = df.ephysFN[rowi]
        df_sort.expControlFN[Expi] = df.expControlFN[rowi]
        if "Stimuli" in df.stimuli[rowi]:
            df_sort.stimuli[Expi] = df.stimuli[rowi]
        else:
            df_sort.stimuli[Expi] = ""
            stimuli_miss_cnt += 1
            # print out info for further examination
            print("\nExp %d\t %s\t %s" % (Expi, df.ephysFN[rowi], df.expControlFN[rowi]))
            print(df.stimuli[rowi:nextrow].str.cat(sep=""))
            if ("Abort" in df_sort.comments[Expi]) or ("abort" in df_sort.comments[Expi]):
                print("Do aborted! No worry.")
    print(stimuli_miss_cnt, "stimuli missing")
    #%%
    df_sort.to_excel(out_excel,index=False)
    return df_sort

In [37]:
(df_formal.expControlFN==name).nonzero()[0][0]

141

In [57]:
def concat_table(df_old, df_new, addexplabel=None, out_path=None):
    if isinstance(df_old,str):
        out_path = df_old
        df_old = pd.read_excel(df_old)
    if isinstance(df_new,str):
        df_new = pd.read_excel(df_new)
    # Check if the experiments in the new datatable has been recorded
    break_flag = False
    for name in df_new.expControlFN:
        if (df_old.expControlFN==name).any():
            print("%s  has been recorded in the excel index %d, please check"%(name, (df_old.expControlFN==name).nonzero()[0][0]))
            break_flag = True
    if break_flag:
        raise ValueError
    if addexplabel is not None:
        df_new.Exp_collection[:] = addexplabel
    df_cat = pd.concat([df_old,df_new], axis=0, ignore_index=True)
    df_cat.to_excel(out_path,index=False)
    return df_cat

def sort_merge_table(df_sort, addexplabel=None):
    Animal_strs = ["Alfa", "Beto"]
    df_paths = [r"S:\Exp_Record_Alfa.xlsx", r"S:\ExpSpecTable_Augment.xlsx"]
    if isinstance(df_sort,str):
        df_sort = pd.read_excel(df_sort)
    # loop through animal name and sort corresponding exp to the collection
    for animal, out_path in zip(Animal_strs, df_paths):
        print("Sort out exp for %s, adding "%animal)
        df_old = pd.read_excel(out_path) # load the old exp collection
        id_col = []
        for idx in df_sort.index:
            name = df_sort.expControlFN[idx]
            if animal in name:
                if (df_old.expControlFN==name).any():
                    print("%s  has been recorded in the excel index %d, please check. Skipping."%(name, (df_old.expControlFN==name).nonzero()[0][0]))
                else:
                    id_col.append(idx)
        df_ftr = df_sort.iloc[id_col].copy()
        print(df_ftr.expControlFN)
        if addexplabel is not None:
            df_ftr.Exp_collection[:] = addexplabel
        df_cat = pd.concat([df_old, df_ftr], axis=0, ignore_index=True)
        df_cat.to_excel(out_path,index=False) # write to the excel of all old experiments 
    return
# pd.concat([df_formal,df_sort], axis=0, ignore_index=True)

In [56]:
df_sort.expControlFN

0             200318_Beto_rfMapper_basic
1          200318_Beto_rfMapper_basic(1)
2          200318_Beto_rfMapper_basic(2)
3     200318_Beto_generate_integrated(1)
4     200318_Beto_generate_integrated(2)
5     200318_Beto_generate_integrated(3)
6     200318_Beto_generate_integrated(4)
7     200318_Beto_generate_integrated(5)
8             200318_Alfa_rfMapper_basic
9          200318_Alfa_rfMapper_basic(1)
10       200318_Alfa_generate_integrated
11    200318_Alfa_generate_integrated(1)
12    200318_Alfa_generate_integrated(2)
13    200318_Alfa_generate_integrated(3)
14    200318_Alfa_generate_integrated(4)
Name: expControlFN, dtype: object

In [33]:
df_sort.expControlFN[1]

'200318_Beto_rfMapper_basic(1)'

In [25]:
Animal = "Both"#"Beto" # "Alfa" "ALfa"
df_sort = process_concat_cells(r"D:\ExpRecord_tmp.xlsx", "D:\ExpRecord_out.xlsx", animal=Animal)


Exp 0	 Beto-18032020-001	 200318_Beto_rfMapper_basic
001 RFMapping 11:
-8:8:8 
Carlos' huge image test
Completed

Exp 1	 Beto-18032020-002	 200318_Beto_rfMapper_basic(1)
002 RFMapping starts 11:35
-8:2:8
Completed

Exp 2	 Beto-18032020-003	 200318_Beto_rfMapper_basic(2)
003 RFMapping starts 11:49
-4:1:4
8 mins similar RF position
Completed

Exp 3	 Beto-18032020-004	 200318_Beto_generate_integrated(1)
004 Generate Integrated
24 [-1.5 1.5 ] 4 1 CMAES
Trying out basic optimizer, see if it can evolve!
Seems work, Evolving very fast.
Seems plateau really fast as well around 10 gens. Only 6 mins! Wow. Maybe 20 blocks
Why Stuck so early? Gets some local maxima? 
Completed

Exp 4	 Beto-18032020-005	 200318_Beto_generate_integrated(2)
005 Generate Integrated
24 [-1.5 1.5 ] 4 1 ZOHA_Sphere_lr_euclid
24 [-1.5 1.5 ] 4 1 ZOHA_Sphere_lr_euclid_RD
Test the reduced Dimension Comparison.
Use the inverse decay exploration range. lr=1.5
Inverse decay's initial learning rate is too large, should make it 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

0 stimuli missing


In [27]:
df_sort

,ephysFN,expControlFN,stimuli,comments,Exp_collection,Expi,pref_chan,stim_size
0,Beto-18032020-001,200318_Beto_rfMapper_basic,N:\Stimuli\2019-06-RF-mapping\2020-03-18-Beto,001 RFMapping 11:\n-8:8:8 \nCarlos' huge image...,NaN,NaN,NaN,NaN
1,Beto-18032020-002,200318_Beto_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-03-18-Beto,002 RFMapping starts 11:35\n-8:2:8\nCompleted,NaN,NaN,NaN,NaN
2,Beto-18032020-003,200318_Beto_rfMapper_basic(2),N:\Stimuli\2019-06-RF-mapping\2020-03-18-Beto,003 RFMapping starts 11:49\n-4:1:4\n8 mins sim...,NaN,NaN,NaN,NaN
3,Beto-18032020-004,200318_Beto_generate_integrated(1),N:\Stimuli\2019-12-Evolutions\2020-03-18-Beto-...,004 Generate Integrated\n24 [-1.5 1.5 ] 4 1 CM...,NaN,NaN,NaN,NaN
4,Beto-18032020-005,200318_Beto_generate_integrated(2),N:\Stimuli\2019-12-Evolutions\2020-03-18-Beto-...,005 Generate Integrated\n24 [-1.5 1.5 ] 4 1 ZO...,NaN,NaN,NaN,NaN
5,Beto-18032020-006,200318_Beto_generate_integrated(3),N:\Stimuli\2019-12-Evolutions\2020-03-18-Beto-...,006 Generate Integrated\n24 [-1.5 1.5 ] 4 1 ZO...,NaN,NaN,NaN,NaN
6,Beto-18032020-007,200318_Beto_generate_integrated(4),N:\Stimuli\2019-12-Evolutions\2020-03-18-Beto-...,007 Generate Integrated starts 13:30\n64 [-1.5...,NaN,NaN,NaN,NaN
7,Beto-18032020-008,200318_Beto_generate_integrated(5),N:\Stimuli\2019-12-Evolutions\2020-03-18-Beto-...,008 Generate Integrated starts 14:10\n64 [-1.5...,NaN,NaN,NaN,NaN
8,Alfa64chan-18032020-001,200318_Alfa_rfMapper_basic,N:\Stimuli\2019-06-RF-mapping\2020-03-18-Alfa,"001 Rf mapper 8, [8 8] start at 1:06\n10 block...",NaN,NaN,NaN,NaN
9,Alfa64chan-18032020-002,200318_Alfa_rfMapper_basic(1),N:\Stimuli\2019-06-RF-mapping\2020-03-18-Alfa,002 Rf mapper\n002 2 [8 8] start at 1:10\nCh 1...,NaN,NaN,NaN,NaN


In [58]:
sort_merge_table(df_sort, addexplabel="ReducDimen_Evol")

Sort out exp for Alfa, adding 
8             200318_Alfa_rfMapper_basic
9          200318_Alfa_rfMapper_basic(1)
10       200318_Alfa_generate_integrated
11    200318_Alfa_generate_integrated(1)
12    200318_Alfa_generate_integrated(2)
13    200318_Alfa_generate_integrated(3)
14    200318_Alfa_generate_integrated(4)
Name: expControlFN, dtype: object


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



Sort out exp for Beto, adding 
200318_Beto_rfMapper_basic  has been recorded in the excel index 177, please check. Skipping.
200318_Beto_rfMapper_basic(1)  has been recorded in the excel index 178, please check. Skipping.
200318_Beto_rfMapper_basic(2)  has been recorded in the excel index 179, please check. Skipping.
200318_Beto_generate_integrated(1)  has been recorded in the excel index 180, please check. Skipping.
200318_Beto_generate_integrated(2)  has been recorded in the excel index 181, please check. Skipping.
200318_Beto_generate_integrated(3)  has been recorded in the excel index 182, please check. Skipping.
200318_Beto_generate_integrated(4)  has been recorded in the excel index 183, please check. Skipping.
200318_Beto_generate_integrated(5)  has been recorded in the excel index 184, please check. Skipping.
Series([], Name: expControlFN, dtype: object)


In [12]:
if Animal == "Beto":
    table_old = r"S:\ExpSpecTable_Augment.xlsx"
elif Animal == "Alfa":
    table_old = r"S:\Exp_Record_Alfa.xlsx"
df_cat = concat_table(table_old, r"D:\ExpRecord_out.xlsx", addexplabel="ReducDimen_Evol") # "Optim_tuning"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [22]:
df_sort.Exp_collection[:] = "SUHash"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
!S:\ExpSpecTable_Augment.xlsx

In [30]:
!S:\Exp_Record_Alfa.xlsx

The process cannot access the file because it is being used by another process.


## Matlab Data Processing

In [5]:
format compact

In [1]:
Set_Path;

Drive already SUBSTed 

ans =

     1




In [4]:
winopen("S:\ExpSpecTable_Augment.xlsx")

In [2]:
open("Project_Manifold_Beto_loadRaw")

In [3]:
open("Evol_Optimizer_Cmp")